In [88]:
import pandas as pd
import numpy as np

import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score,\
recall_score, roc_auc_score, confusion_matrix, classification_report

warnings.filterwarnings(action='ignore')
plt.rc('font', family='Noto Sans CJK JP')

In [89]:
df_all = pd.read_csv('./new_df_chaju.csv', engine = 'python')
df_all.head()

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,Y,0,N
1,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,N,2,N
2,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,N,2,N
3,5,C-1006,C06,521,여성,59,50대,판매/서비스직,고등학교 졸업,자가,아파트,기타(이혼/별거/사별),300-399만원,100-199만원,99만원 이하,N,2,N
4,7,C-1008,C06,528,남성,35,30대,판매/서비스직,대학교 졸업,전세,아파트,기혼,500-599만원,300-399만원,99만원 이하,N,2,Y


In [90]:
df_all.columns

Index(['Unnamed: 0', 'chaju_id', 'credit_class', 'score_risk', 'gender', 'age',
       'agegroup', 'job', 'edu', 'dwell_type', 'house_type', 'marriage',
       'income_home_grp', 'income_ind_grp', 'expenditure_grp', 'child_cd',
       'no_child', 'flag_priv_biz_cd'],
      dtype='object')

In [91]:
add_df = pd.read_csv('./df_all_cnt포함.csv')
add_df.head()

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,loan_amt,over_period,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,loan_count,loan_count1,overdue_count,overdue_count1
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,70300000.0,2.0,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108,9.0,9.0,1.0,1.0
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,모름/무응답,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,3150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000,2.0,2.0,0.0,0.0
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,391000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000,5.0,5.0,0.0,0.0
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,모름/무응답,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,590000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000,2.0,2.0,0.0,0.0
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,367200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000,4.0,4.0,0.0,0.0


In [92]:
add_df.columns

Index(['Unnamed: 0', 'chaju_id', 'credit_class', 'score_risk', 'gender', 'age',
       'agegroup', 'job', 'edu', 'dwell_type', 'house_type', 'marriage',
       'income_home_grp', 'income_ind_grp', 'expenditure_grp', 'child_cd',
       'no_child', 'flag_priv_biz_cd', 'card_count', 'card_period',
       'loan_period', 'Pcard_loan_amt', 'loan_amt', 'over_period',
       'Pcard_over_amt', 'over_amt', 'over_normal_amt', 'over_special_amt',
       'over_s98case_amt', '연체율', 'loan_amt_final', '연체율특채반영',
       'loan_amt_final2', '연체율S98반영', 'loan_count', 'loan_count1',
       'overdue_count', 'overdue_count1'],
      dtype='object')

In [93]:
add_df['income_ind_grp1'] = df_all['income_ind_grp']
add_df

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,loan_amt,over_period,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,loan_count,loan_count1,overdue_count,overdue_count1,income_ind_grp1
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,70300000.0,2.0,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108,9.0,9.0,1.0,1.0,300-399만원
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,모름/무응답,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,3150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000,2.0,2.0,0.0,0.0,500-599만원
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,391000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000,5.0,5.0,0.0,0.0,500-599만원
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,모름/무응답,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,590000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000,2.0,2.0,0.0,0.0,100-199만원
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,367200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000,4.0,4.0,0.0,0.0,300-399만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,Y,0,N,1.0,25.0,29.0,0.0,68000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,68000000.0,0.000000,68000000.0,0.000000,2.0,2.0,0.0,0.0,200-299만원
1980,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,N,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,99만원 이하
1981,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,N,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,200-299만원
1982,1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,N,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,300-399만원


In [94]:
add_df.drop(['Unnamed: 0','income_ind_grp'], axis = 1, inplace = True)
add_df.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,loan_amt,over_period,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,loan_count,loan_count1,overdue_count,overdue_count1,income_ind_grp1
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,70300000.0,2.0,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108,9.0,9.0,1.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,3150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000,2.0,2.0,0.0,0.0,500-599만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,391000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000,5.0,5.0,0.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,590000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000,2.0,2.0,0.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,367200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000,4.0,4.0,0.0,0.0,300-399만원


In [95]:
add_df.rename(columns = {'income_ind_grp1' : 'income_ind_grp'} , inplace = True)
add_df.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,loan_amt,over_period,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,연체율S98반영,loan_count,loan_count1,overdue_count,overdue_count1,income_ind_grp
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,70300000.0,2.0,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108,9.0,9.0,1.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,3150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000,2.0,2.0,0.0,0.0,500-599만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,391000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000,5.0,5.0,0.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,590000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000,2.0,2.0,0.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,367200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000,4.0,4.0,0.0,0.0,300-399만원


In [96]:
add_df.isnull().sum()

chaju_id              0
credit_class          0
score_risk            0
gender                0
age                   0
agegroup              0
job                   0
edu                   0
dwell_type            0
house_type            0
marriage              0
income_home_grp       0
expenditure_grp       0
child_cd              0
no_child              0
flag_priv_biz_cd      0
card_count          325
card_period         325
loan_period           0
Pcard_loan_amt        0
loan_amt              0
over_period           0
Pcard_over_amt        0
over_amt              0
over_normal_amt       0
over_special_amt      0
over_s98case_amt      0
연체율                   0
loan_amt_final        0
연체율특채반영             555
loan_amt_final2       0
연체율S98반영            555
loan_count            0
loan_count1           0
overdue_count         0
overdue_count1        0
income_ind_grp        0
dtype: int64

In [97]:
add_df = add_df.fillna(0)

In [98]:
add_df.isnull().sum()

chaju_id            0
credit_class        0
score_risk          0
gender              0
age                 0
agegroup            0
job                 0
edu                 0
dwell_type          0
house_type          0
marriage            0
income_home_grp     0
expenditure_grp     0
child_cd            0
no_child            0
flag_priv_biz_cd    0
card_count          0
card_period         0
loan_period         0
Pcard_loan_amt      0
loan_amt            0
over_period         0
Pcard_over_amt      0
over_amt            0
over_normal_amt     0
over_special_amt    0
over_s98case_amt    0
연체율                 0
loan_amt_final      0
연체율특채반영             0
loan_amt_final2     0
연체율S98반영            0
loan_count          0
loan_count1         0
overdue_count       0
overdue_count1      0
income_ind_grp      0
dtype: int64

In [99]:
add_df.rename(columns = {'연체율S98반영' : 'del_rate'} , inplace = True)
add_df.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,loan_amt,over_period,Pcard_over_amt,over_amt,over_normal_amt,over_special_amt,over_s98case_amt,연체율,loan_amt_final,연체율특채반영,loan_amt_final2,del_rate,loan_count,loan_count1,overdue_count,overdue_count1,income_ind_grp
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,70300000.0,2.0,0.0,5700000.0,5700000.0,0.0,0.0,8.108108,70300000.0,8.108108,70300000.0,8.108108,9.0,9.0,1.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,3150000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,3150000.0,0.000000,3150000.0,0.000000,2.0,2.0,0.0,0.0,500-599만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,391000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,391000000.0,0.000000,391000000.0,0.000000,5.0,5.0,0.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,590000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,590000.0,0.000000,590000.0,0.000000,2.0,2.0,0.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,367200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,367200000.0,0.000000,367200000.0,0.000000,4.0,4.0,0.0,0.0,300-399만원


In [100]:
add_df.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'loan_amt', 'over_period', 'Pcard_over_amt', 'over_amt',
       'over_normal_amt', 'over_special_amt', 'over_s98case_amt', '연체율',
       'loan_amt_final', '연체율특채반영', 'loan_amt_final2', 'del_rate',
       'loan_count', 'loan_count1', 'overdue_count', 'overdue_count1',
       'income_ind_grp'],
      dtype='object')

In [101]:
add_df.drop(['loan_amt','over_normal_amt','over_special_amt','over_s98case_amt','연체율','loan_amt_final','연체율특채반영'], axis = 1, inplace = True)
add_df.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'loan_count1', 'overdue_count',
       'overdue_count1', 'income_ind_grp'],
      dtype='object')

# loan, overdue count drop 여부

In [102]:
# count1, count 비교
def GetDifferences(df1, df2):
    df = pd.concat([df1, df2]).reset_index(drop=True)
    idx = [diff[0] for diff in df.groupby(list(df)).groups.values() if len(diff) == 1]
    return df.reindex(idx)

In [103]:
GetDifferences(add_df['overdue_count'], add_df['overdue_count1'])

Series([], dtype: float64)

In [104]:
GetDifferences(add_df['loan_count'], add_df['loan_count1'])

Series([], dtype: float64)

In [105]:
add_df.drop(['overdue_count1','loan_count1'], axis = 1, inplace = True)

In [106]:
add_df.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,Y,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,N,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,500-599만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,N,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,Y,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,N,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,300-399만원


In [107]:
add_df.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'overdue_count', 'income_ind_grp'],
      dtype='object')

In [108]:
add_df.to_csv('./final_df_all.csv', index = False)

# 오류난 부분 해결하기

In [109]:
right = pd.read_csv('./new_chaju_df.csv')
right.drop('Unnamed: 0', axis = 1, inplace = True)
right

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,Y,0,N
1,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,N,2,N
2,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,N,2,N
3,C-1006,C06,521,여성,59,50대,판매/서비스직,고등학교 졸업,자가,아파트,기타(이혼/별거/사별),300-399만원,100-199만원,99만원 이하,N,2,N
4,C-1008,C06,528,남성,35,30대,판매/서비스직,대학교 졸업,전세,아파트,기혼,500-599만원,300-399만원,99만원 이하,N,2,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2986,C04,368,여성,46,40대,전업주부,고등학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,100-199만원,N,2,N
1980,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,99만원 이하,99만원 이하,Y,0,N
1981,C-2991,C06,528,여성,39,30대,전업주부,고등학교 졸업,자가,아파트,기혼,400-499만원,200-299만원,200-299만원,N,2,N
1982,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,N,1,Y


In [110]:
right.sort_values(by = 'chaju_id',ascending=True,inplace = True)

In [111]:
right

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,Y,0,N
1380,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,N,3,Y
1,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,N,2,N
1381,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,Y,0,N
2,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,N,2,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,Y,0,N
1377,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,N,2,N
1378,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,N,2,N
1379,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,N,2,N


In [112]:
final = pd.read_csv('./final_df_all_ver0.4.csv')
final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,200-299만원
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,100-199만원
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,300-399만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0,200-299만원
1425,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0,100-199만원
1426,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0,200-299만원
1427,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,200-299만원


In [146]:
total_final = pd.read_csv('./final_df_all_ver0.2.csv')
total_final

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,350
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,250
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,550
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,150
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,250
1980,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,350
1981,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,250
1982,1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,350


In [113]:
right[['income_ind_grp']].reset_index(drop=True, inplace=True)
# right[['income_ind_grp']].drop('index', axis = 1, inplace = True)
right[['income_ind_grp']]

,income_ind_grp
0,300-399만원
1380,100-199만원
1,500-599만원
1381,200-299만원
2,500-599만원
...,...
1376,300-399만원
1377,300-399만원
1378,100-199만원
1379,300-399만원


In [114]:
right[['income_ind_grp']].to_csv('./income_ind.csv',index = False)

In [115]:
ind = pd.read_csv('./income_ind.csv')
ind

,income_ind_grp
0,300-399만원
1,100-199만원
2,500-599만원
3,200-299만원
4,500-599만원
...,...
1979,300-399만원
1980,300-399만원
1981,100-199만원
1982,300-399만원


In [147]:
total_final['income_ind_grp1'] = ind
total_final

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp,income_ind_grp1
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,350,300-399만원
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,250,100-199만원
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,550,500-599만원
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,150,200-299만원
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,350,500-599만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,250,300-399만원
1980,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,350,300-399만원
1981,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,250,100-199만원
1982,1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,350,300-399만원


In [148]:
total_final.drop('income_ind_grp',axis = 1, inplace =  True)
total_final.rename(columns = {'income_ind_grp1' : 'income_ind_grp'}, inplace = True)
total_final

,Unnamed: 0,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp
0,0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원
1,1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,100-199만원
2,2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원
3,3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,200-299만원
4,4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,500-599만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,300-399만원
1980,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,300-399만원
1981,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,100-199만원
1982,1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,300-399만원


In [150]:
total_final.columns

Index(['Unnamed: 0', 'chaju_id', 'credit_class', 'score_risk', 'gender', 'age',
       'agegroup', 'job', 'edu', 'dwell_type', 'house_type', 'marriage',
       'income_home_grp', 'expenditure_grp', 'child_cd', 'no_child',
       'flag_priv_biz_cd', 'card_count', 'card_period', 'loan_period',
       'Pcard_loan_amt', 'over_period', 'Pcard_over_amt', 'over_amt',
       'loan_amt_final2', 'del_rate', 'loan_count', 'overdue_count',
       'income_ind_grp'],
      dtype='object')

In [151]:
total_final = total_final[['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp','income_ind_grp',
      'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'overdue_count']]
total_final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [120]:
import pandas as pd
print("pandas version: ", pd.__version__)
pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

pandas version:  1.2.4


In [153]:
total_final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [152]:
total_final.to_csv('./final_df_all_ver0.6(전체).csv', index = False)

In [124]:
thin = pd.read_csv('./df_thin_1811.csv', encoding = 'cp949')
thin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp,real_period
0,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,400-499만원,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200-299만원,115.0
1,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,400-499만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100-199만원,99.0
2,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,400-499만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400-499만원,119.0
3,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,500-599만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200-299만원,110.0
4,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,300-399만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300-399만원,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,400-499만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,400-499만원,141.0
551,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600-699만원,93.0
552,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300-399만원,80.0
553,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200-299만원,181.0


In [165]:
thin.columns

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'overdue_count', 'income_ind_grp',
       'real_period'],
      dtype='object')

# 씬파일러 부분 incom_ind 해결 

In [154]:
total_final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,350,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [155]:
thin_chaju = list(thin['chaju_id'])
thin_chaju

['C-1010',
 'C-1012',
 'C-1014',
 'C-1016',
 'C-1018',
 'C-1026',
 'C-1028',
 'C-1032',
 'C-1033',
 'C-1038',
 'C-1051',
 'C-1054',
 'C-1059',
 'C-1060',
 'C-1062',
 'C-1065',
 'C-1069',
 'C-1073',
 'C-1074',
 'C-1079',
 'C-1080',
 'C-1082',
 'C-1087',
 'C-1089',
 'C-1090',
 'C-1092',
 'C-1095',
 'C-1096',
 'C-1098',
 'C-1101',
 'C-1106',
 'C-1107',
 'C-1114',
 'C-1121',
 'C-1123',
 'C-1128',
 'C-1131',
 'C-1133',
 'C-1136',
 'C-1150',
 'C-1151',
 'C-1152',
 'C-1153',
 'C-1158',
 'C-1159',
 'C-1161',
 'C-1166',
 'C-1167',
 'C-1169',
 'C-1174',
 'C-1175',
 'C-1178',
 'C-1185',
 'C-1188',
 'C-1190',
 'C-1194',
 'C-1195',
 'C-1198',
 'C-1203',
 'C-1206',
 'C-1216',
 'C-1219',
 'C-1222',
 'C-1226',
 'C-1230',
 'C-1232',
 'C-1233',
 'C-1235',
 'C-1236',
 'C-1242',
 'C-1245',
 'C-1248',
 'C-1249',
 'C-1251',
 'C-1258',
 'C-1259',
 'C-1260',
 'C-1270',
 'C-1272',
 'C-1274',
 'C-1275',
 'C-1281',
 'C-1282',
 'C-1291',
 'C-1292',
 'C-1294',
 'C-1297',
 'C-1302',
 'C-1303',
 'C-1305',
 'C-1312',

In [169]:
idx_drop = total_final[(total_final['over_amt'] == 0) & (total_final['loan_amt_final2'] == 0)]
idx_drop

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,450,99만원 이하,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,450,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,450,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,550,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,350,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,450,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,650,99만원 이하,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
idx_drop['real_period'] = thin['real_period']
idx_drop

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,real_period
9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,450,99만원 이하,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0
11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,450,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,157.0
13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,450,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194.0
15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,550,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145.0
17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,350,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,450,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,650,99만원 이하,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,350,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,550,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [177]:
def change(y):
    if y == 550:
        won = '500-599만원'
    elif y == 350:
        won = '300-399만원'
    elif y == 450:
        won = '400-499만원'
    elif y == 650:
        won = '600-699만원'
    elif y == 250:
        won = '200-299만원'
    elif y == 750:
        won = '700-799만원'
    elif y == 150:
        won = '100-199만원'
    elif y == 850:
        won = '800-899만원'
    elif y == 50:
        won = '99만원 이하'
    elif y == 1050:
        won = '1000만원 이상'
    else:
        won = '900-999만원'
    return won

idx_drop['income_home_grp'] = idx_drop['income_home_grp'].apply(lambda y: change(y))
# df_all['income_ind_grp'] = df_all['income_ind_grp'].apply(lambda y: change(y))
idx_drop

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,real_period
9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,400-499만원,99만원 이하,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,61.0
11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,400-499만원,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,157.0
13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,400-499만원,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,194.0
15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,500-599만원,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,145.0
17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,300-399만원,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,400-499만원,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,99만원 이하,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [171]:
idx_drop.to_csv('./final_df_all_ver0.7(씬파일러).csv', index = False)

In [175]:
non_thin1 = total_final[(total_final['over_amt'] != 0) | (total_final['loan_amt_final2'] != 0)]
non_thin1.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0


In [176]:
non_thin1.shape

(1429, 28)

# final_df_all_ver0.6(전체) 수정

## home grp 수정

In [178]:
final = pd.read_csv('./final_df_all_ver0.6(전체).csv')
final.head()

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,950,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,250,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,550,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,550,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,550,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0


In [179]:
def change(y):
    if y == 550:
        won = '500-599만원'
    elif y == 350:
        won = '300-399만원'
    elif y == 450:
        won = '400-499만원'
    elif y == 650:
        won = '600-699만원'
    elif y == 250:
        won = '200-299만원'
    elif y == 750:
        won = '700-799만원'
    elif y == 150:
        won = '100-199만원'
    elif y == 850:
        won = '800-899만원'
    elif y == 50:
        won = '99만원 이하'
    elif y == 1050:
        won = '1000만원 이상'
    else:
        won = '900-999만원'
    return won

final['income_home_grp'] = final['income_home_grp'].apply(lambda y: change(y))
# df_all['income_ind_grp'] = df_all['income_ind_grp'].apply(lambda y: change(y))
final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [180]:
# home < ind인 경우 확인
cnt = 0
a = []
for i in range(len(final)):
    if final['income_home_grp'][i] < final['income_ind_grp'][i]:
        a.append(i)
        cnt += 1

In [181]:
print(a)
print(cnt)

[9, 24, 32, 76, 111, 123, 139, 148, 157, 158, 161, 168, 171, 180, 195, 204, 205, 225, 231, 238, 241, 256, 258, 297, 303, 310, 311, 312, 342, 354, 357, 395, 397, 400, 405, 407, 418, 438, 439, 461, 469, 482, 508, 547, 553, 570, 573, 574, 603, 607, 646, 655, 667, 672, 698, 741, 754, 772, 784, 788, 801, 811, 814, 823, 850, 868, 874, 905, 954, 987, 999, 1023, 1049, 1054, 1057, 1059, 1066, 1067, 1086, 1106, 1109, 1112, 1123, 1138, 1152, 1170, 1202, 1212, 1225, 1251, 1267, 1272, 1310, 1325, 1328, 1347, 1394, 1444, 1445, 1464, 1472, 1484, 1500, 1523, 1545, 1546, 1573, 1590, 1614, 1670, 1671, 1711, 1738, 1745, 1758, 1779, 1788, 1797, 1812, 1824, 1833, 1850, 1854, 1869, 1883, 1903, 1914, 1917, 1920, 1942, 1971]
131


In [182]:
# home = ind 만들기
for i in range(len(df_all)):
    if final['income_home_grp'][i] < final['income_ind_grp'][i]:
        final['income_ind_grp'][i] = final['income_home_grp'][i]

        

In [183]:
# home < ind인 경우 확인
cnt = 0
a = []
for i in range(len(final)):
    if final['income_home_grp'][i] < final['income_ind_grp'][i]:
        a.append(i)
        cnt += 1
        
print(a)
print(cnt)

[]
0


In [184]:
final.to_csv('./final_df_all_ver0.6(전체).csv', index = False)

## 씬파일러, 비씬파일러 구분

In [213]:
final

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1982,C-2999,C04,398,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,2.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [214]:
final_thin = final[(final['over_amt'] == 0) & (final['loan_amt_final2'] == 0)]
final_nthin = final.drop(final_thin.index, axis = 0)

In [215]:
final_thin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,400-499만원,400-499만원,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,400-499만원,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,400-499만원,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,500-599만원,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,300-399만원,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,400-499만원,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,600-699만원,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
print(thin.columns)
print(len(thin.columns))

Index(['chaju_id', 'credit_class', 'score_risk', 'gender', 'age', 'agegroup',
       'job', 'edu', 'dwell_type', 'house_type', 'marriage', 'income_home_grp',
       'expenditure_grp', 'child_cd', 'no_child', 'flag_priv_biz_cd',
       'card_count', 'card_period', 'loan_period', 'Pcard_loan_amt',
       'over_period', 'Pcard_over_amt', 'over_amt', 'loan_amt_final2',
       'del_rate', 'loan_count', 'overdue_count', 'income_ind_grp',
       'real_period'],
      dtype='object')
29


In [217]:
thin.shape

(555, 29)

In [241]:
real_period = thin[['real_period']]
real_period

,real_period
0,115.0
1,99.0
2,119.0
3,110.0
4,124.0
...,...
550,141.0
551,93.0
552,80.0
553,181.0


In [245]:
final_thin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,400-499만원,400-499만원,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,400-499만원,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,400-499만원,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,500-599만원,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,300-399만원,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,400-499만원,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,600-699만원,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [250]:
final_thin_1.drop('index', axis = 1, inplace = True)

In [247]:
# 인덱스 맞춰줘야 값이 제대로 들어감. 0부터 시작해서 대입
final_thin.reset_index(drop=False, inplace = True)

In [248]:
final_thin_1 = pd.concat([final_thin, real_period] , axis = 1)
final_thin_1

,index,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,real_period
0,9,C-1010,C05,430,남성,22,20대,학생(재수생포함),대학교 재학,전세,단독주택,미혼,400-499만원,400-499만원,100-199만원,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,115.0
1,11,C-1012,C04,381,여성,52,50대,기능/노무직,고등학교 졸업,전세,아파트,기혼,400-499만원,100-199만원,99만원 이하,N,0,N,3.0,62.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.0
2,13,C-1014,C05,405,여성,26,20대,사무직,대학교 졸업,월세,오피스텔,미혼,400-499만원,200-299만원,99만원 이하,N,0,N,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,119.0
3,15,C-1016,C04,341,여성,56,50대,전업주부,대학교 졸업,자가,단독주택,기혼,500-599만원,200-299만원,100-199만원,Y,2,N,5.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0
4,17,C-1018,C05,451,여성,53,50대,판매/서비스직,고등학교 졸업,자가,연립/다세대/빌라,기혼,300-399만원,100-199만원,100-199만원,Y,1,N,6.0,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,124.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,1958,C-2974,C05,401,여성,51,50대,기능/노무직,고등학교 졸업,자가,단독주택,기혼,400-499만원,100-199만원,100-199만원,Y,2,N,3.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,141.0
551,1971,C-2988,C03,297,남성,70,70대,학생(재수생포함),대학교 재학,자가,아파트,미혼,600-699만원,600-699만원,99만원 이하,N,0,N,1.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93.0
552,1980,C-2997,C04,308,남성,37,30대,사무직,대학교 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0
553,1981,C-2998,C04,371,여성,50,50대,기능/노무직,고등학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,100-199만원,Y,2,N,5.0,104.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,181.0


In [252]:
final_thin_1.to_csv('./final_df_all_ver0.7(씬파일러).csv', index = False)

In [253]:
final_nthin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1976,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,200-299만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0
1977,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0
1978,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0
1979,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0


In [256]:
nonthin = pd.read_csv('./df_nthin_1802.csv', encoding = 'cp949')
nonthin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp,real_period
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원,222.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,200-299만원,76.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원,234.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,100-199만원,90.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,300-399만원,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0,200-299만원,NaN
1425,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0,100-199만원,NaN
1426,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0,200-299만원,139.0
1427,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,200-299만원,220.0


In [259]:
nonthin['real_period'].fillna(0, inplace = True)
nonthin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,income_ind_grp,real_period
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,300-399만원,222.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,200-299만원,76.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,500-599만원,234.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,100-199만원,90.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,300-399만원,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0,200-299만원,0.0
1425,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0,100-199만원,0.0
1426,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0,200-299만원,139.0
1427,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,200-299만원,220.0


In [260]:
nonthin['real_period'].isnull().sum()

0

In [261]:
final_nthin.reset_index(drop=True, inplace = True)
final_nthin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,200-299만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0
1425,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0
1426,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0
1427,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0


In [262]:
final_nthin['real_period'] = nonthin['real_period']
final_nthin

,chaju_id,credit_class,score_risk,gender,age,agegroup,job,edu,dwell_type,house_type,marriage,income_home_grp,income_ind_grp,expenditure_grp,child_cd,no_child,flag_priv_biz_cd,card_count,card_period,loan_period,Pcard_loan_amt,over_period,Pcard_over_amt,over_amt,loan_amt_final2,del_rate,loan_count,overdue_count,real_period
0,C-1001,C05,478,남성,30,30대,기능/노무직,대학교 졸업,자가,아파트,미혼,900-999만원,300-399만원,100-199만원,N,0,N,4.0,99.0,79.0,0.0,2.0,0.0,5700000.0,70300000.0,8.108108,9.0,1.0,222.0
1,C-1002,C05,484,여성,66,60대,전업주부,초등학교 졸업,자가,아파트,기혼,200-299만원,100-199만원,99만원 이하,Y,3,Y,6.0,126.0,6.0,350000.0,0.0,0.0,0.0,3150000.0,0.000000,2.0,0.0,76.0
2,C-1003,C05,478,남성,43,40대,판매/서비스직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,100-199만원,Y,2,N,2.0,50.0,79.0,0.0,0.0,0.0,0.0,391000000.0,0.000000,5.0,0.0,234.0
3,C-1004,C05,445,남성,28,20대,무직/기타,대학교 졸업,자가,아파트,미혼,500-599만원,200-299만원,99만원 이하,N,0,N,1.0,25.0,5.0,0.0,0.0,0.0,0.0,590000.0,0.000000,2.0,0.0,90.0
4,C-1005,C05,421,남성,52,50대,행정관리/전문직,대학교 졸업,자가,아파트,기혼,500-599만원,500-599만원,200-299만원,Y,2,N,3.0,75.0,40.0,0.0,0.0,0.0,0.0,367200000.0,0.000000,4.0,0.0,264.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1424,C-2993,C04,398,여성,35,30대,사무직,전문대학 졸업,전세,아파트,기혼,500-599만원,200-299만원,99만원 이하,Y,1,N,0.0,0.0,17.0,0.0,0.0,0.0,0.0,7900000.0,0.000000,2.0,0.0,0.0
1425,C-2994,C05,468,남성,42,40대,기능/노무직,전문대학 졸업,전세,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,2,N,0.0,0.0,25.0,0.0,6.0,0.0,500000.0,1360000.0,36.764706,1.0,1.0,0.0
1426,C-2995,C06,528,여성,39,30대,전업주부,전문대학 졸업,자가,아파트,기혼,300-399만원,300-399만원,100-199만원,Y,1,Y,5.0,118.0,150.0,16600000.0,0.0,0.0,0.0,61340000.0,0.000000,17.0,0.0,139.0
1427,C-2996,C05,448,남성,41,40대,사무직,대학교 졸업,자가,단독주택,미혼,300-399만원,300-399만원,100-199만원,N,0,N,1.0,25.0,29.0,0.0,0.0,0.0,0.0,68000000.0,0.000000,2.0,0.0,220.0


In [266]:
final_nthin['real_period'].isnull().sum()

0

In [265]:
final_nthin.to_csv('./final_df_all_ver0.7(씬제거).csv', index = False)